#### __Final Project__

#### Python for Computer Vision with OpenCV 

#### Herb Guzman
#### UTSA
#### Fall 2019

Project Assignment: Create a program that can detect a hand, segment the had, and count the number of fingers being held up.
This is designed to work with most webcams, but for this demonstration an Azure Kinect DK device was used.

###### Strategie for Counting Fingers

-  Grab a Range of Interest (ROI)
-  Calculate a running average background value for 60 frames of video (i.e. if rate is 30 frames/sec, then 2 seconds)
-  Once the average value has been found, the hand can enter the ROI
-  Once the hand is in the ROI, detect the change and apply thresholding to isolate the hand and hand segment
-  With the hand detected in the ROI, use a Convex Hull to draw a polygon around the hand
-  With the polygon outlined, calculate the center of the hand
-  Next, use the center of the hand to calculate the angle of the outer points to infer the finger count
   the polygon will have points at each finger.
-  Based on the distance from the center, determine if the finger is extended or not



In [1]:
import cv2
import numpy as np

from sklearn.metrics import pairwise

In [2]:
# GLOBAL VARIABLES

# Initialize the background prior to sensing anything
background = None

# Initialize the accumulated weight to some middle value
accumulated_weight = 0.5

# Default definition for region_of_interest = ROI
# Corners of the rectangle on the screen
# THE INITIAL ROI MAY NEED ADJUSTMENTS

ROI_Top = 30
ROI_Bottom = 400
ROI_Right = 500
ROI_Left = 800


In [3]:
# FUNCTION FOR UPDATING A RUNNING AVERAGE OF THE BACKGROUND VALUES IN
# A RANGE OF INTEREST (ROI).
# This function will allow to detect new objects (the hand) entering and leaving the ROI

def calc_accum_avg(frame, accumulated_weight):
    
    global background
    
    # For the first time...
    # if the background is none, set the background
    # as a copy of the frame being passed into this function
    #
    # Else,
    # if there is something other than none assigned to the 
    # background: use the cv2 function accumulateWeighted
    # where in you pass the source (src), destination (dst) and an
    # alpha value.  This function calculates the weighted sum
    # of the source image and the accumulator destination; then,
    # the destination becomes a running average of a frame sequence.
    # This function essentially accumulates the weight based on the running 
    # average
    # Note that there is no returned value from the accumulateWeighted function
    # we are only doing an update with the accumulated weight
    if background is None: 
        background = frame.copy().astype('float')
        return None
    
    cv2.accumulateWeighted(frame,background,accumulated_weight)


In [4]:
# FUNCTION FOR THRESHOLDING AND CONTOURING
# Segment the hand region in the frame's ROI
# Using thresholding
# Create a "white" hand and create a countour around that.
# Use a minimum value of 25 for a binary threshold
# This will be affected by the uniforminty and colors in the background
# THE THRESHOLD MAY NEED ADJUSTMENT. THIS THRESHOLD VALUE CAN BE USED
# TO CONTROL THE BACKGROUND NOISE
#
# So, threshold_min is the minimum value and is set to 25
# The max value can be set to 255
#
def segment(frame,threshold_min=30):

    # (1) Calculate the absolute difference between the background and
    # the frame passed in to the function
    #
    # (2) Next apply a threshold to this image
    #
    # (3) Next, grab the external contours from the image:
    # To do this use the findContours function of cv2 using a copy of
    # the thresholded image and obtain the EXTERNAL contours of this image
    # Use the "CHAIN_APPROX_SIMPLE" as the method used to calculate the contours
    #
    # (4) Make sure the length of the contours is not zero; that is, the
    # algorithm found some contours. If the contour length is not zero,
    # then the largest external contour is the hand.  This means that if
    # another long object such as a pencil or marker is in the ROI, this 
    # will confuse this function.
    #
    diff = cv2.absdiff(background.astype('uint8'),frame)
    
    ret,thresholded_image = cv2.threshold(diff,threshold_min,255,cv2.THRESH_BINARY)
    
    image,contours,hierarchy = cv2.findContours(thresholded_image.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours) == 0:
        return None
    
    else:
        # Assuming the largest external contour in the ROI is the hand
        # Need to use a "key" to handle the numpy array structure.
        # This will grab the contour with the largest amount of area.
        # This will help discriminate with smaller contours in the background.
        hand_segment = max(contours,key=cv2.contourArea)
        
        return (thresholded_image,hand_segment)

In [5]:
# FUNCTION FOR FINGER COUNTING USING A CONVEX HULL METHOD
# This function counts the fingers held up using the Convex Hull method
# to draw a polygon connecting the points around the most external
# points in a frame.  The way a Convex Hull works is that it will 
# draw a polygon connect the most external points in a data set.
# The way the logic will work for this function is that if the 
# length from the hand's center to each outer point is low, then the
# function will assume the finger is retracted.  Otherwise, if the
# length of approximately the same as the other fingers, then the finger
# will be assumed to be extended.
# Also, the function will need to account for the points coming from the wrist.
# The logic for this process is as follows:
# 
# 1. Calculate the most extreme points, max top, max bottom, max left, max right.
# 2. Use their intersection to estimate the center of the hand
# 3. Calculate the distance from the point furthest away from the center
# 4. Use that distance to create a circle
# 5. Any points outside the circle or close to the edge of the circle will
#    be assumed to be extended fingers.

def count_fingers(thresholded_image,hand_segment):
    
    # Calculate the convex hull for the hand segment contour
    conv_hull = cv2.convexHull(hand_segment)
    
    # Obtain the extreme points from the Hull (top, bottom, right and left)
    # Because of the format of the convex hull output, we need the following steps:
    # According to the OpenCV on contour approximation and Convex Hull using and enclosing polygon
    
    # Obtain the most extreme Top point...
    # grab the first index of the minimum argument of this vector
    # Obtain this as a tuple and assign it to the variable "top"
    # Repeat this idea for the bottom, left, and right-most points:
    
    # Debugging code
    # print ("Val of conv_hull :",conv_hull)
    
    top =    tuple( conv_hull[conv_hull[:, :, 1].argmin()][0] )    # Min
    bottom = tuple( conv_hull[conv_hull[:, :, 1].argmax()][0] )    # Max
    left =   tuple( conv_hull[conv_hull[:, :, 0].argmin()][0] )    # Min
    right =  tuple( conv_hull[conv_hull[:, :, 0].argmax()][0] )    # Max
    
    # Now, calculate the "center" of the hand
    # Note that index 0 are the x coordinates and index 1 are the y coordinates
    # Divide by 2 using two // to make sure the result is an integer
    
    cX = (left[0] + right[0]) // 2
    cY = (top[1] + bottom[1]) // 2
    
    # Distance calculation using the sklearn pairwise function
    # Calculate the euclidean distance between the center and the four outer points
    # Provide a list of the points we need to calculate a distance, call it 'Y'
    # Make sure to only grab the very first item returned (i.e. 0)
    # This returns all the distances

    # Debugging code
    # print ("Val of left :",left)
    # print ("Val of right :",right)
    # print ("Val of top :",top)
    # print ("Val of bottom :",bottom)
    #
    # print ("Val of cX :",cX)
    # print ("Val of cY :",cY)
    
    distance = pairwise.euclidean_distances([(cX,cY)],Y=[left,right,top,bottom])[0]
    
    # Now, determine the maximum distance, which is really the only distances we need
    max_distance = distance.max()
    
    # Now, calculate a circle with a radius of 90% of "max_distance"
    # THE 90% adjustment value can be adjusted in case the hand has very short fingers
    # Make this value an integer; also calculate the circumference of the circle
    radius = int(0.75* max_distance)
    circumference = (2 * np.pi * radius)
    
    # Now, create an ROI
    # First initialize a place holder of the same structure as the thresholded_image with zeros as content
    # Only the X and Y of the thresholded image; we do not need the color channels
    # This data type is an 8-bit integer
    circular_roi = np.zeros(thresholded_image.shape[:2], dtype='uint8')
    
    # Now 'draw' the circular ROI using cv2
    # Use the circular_roi as the image for the circle
    # Use cX and cY for the center points, provide the radius obtained above
    # use a color of 255 and the thickness of the circle as 10 pixels
    cv2.circle(circular_roi,(cX,cY),radius,255,10)
    
    # Use bitwise AND with the circle_roi image as a mask and the thresholded_image
    circular_roi = cv2.bitwise_and(thresholded_image,thresholded_image,mask=circular_roi)
    
    # Now grab all the EXTERNAL contours in the resulting circular_roi, but use a copy of the circular_roi for now
    # FOR THE METHOD, set it equal to chain_approx_none
    image,contours,hierarchy = cv2.findContours(circular_roi.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    
    # Finally, count the number of points/fingers outside the circle
    
    # Initialize the [finger] count to zero
    count = 0
    
    # For every contour in the list of contours, grab the bounding box of the contour
    # with the following conditions:
    # (1) The contour region is not at the bottom of the hand (discriminate the wrist and lower arm)
    #     if it is too far below the center of the hand
    for cnt in contours:
        
        (x,y,w,h) = cv2.boundingRect(cnt)
    
        # Discriminate the wrist: center y plus the 25% of center y distance is greater than the height plus the y value
        # for a given bouding rectangle for the corresponding contour
        out_of_wrist = (cY + (cY*0.25)) > (y+h)
    
        # Check that the number of points around the contour does not exceed the 25% of the circumference of the circular region
        # of interest.  Otherwise, we could be including points completely outside the hand itself.
        # So, define some limit points.
        limit_points = ((circumference*0.25) > cnt.shape[0])
        
        # Now, test for the conditions:
        # (2) If it is outside of the wrist and is not one of the limit points, 
        #     then the point actually belongs to a finger and we can increment the finger count
        if out_of_wrist and limit_points:
            count += 1
        
    return count


In [6]:
# MAIN PROGRAM
#
# This program uses the functions defined above to capture live video from a person's hand and detect how many
# fingers are shown on the screen.

# Grab the video from the default camera
cam = cv2.VideoCapture(1)

# Accumulate 60 frames for background
num_frames = 0

while True:
    
    # Read a frame from the camera
    ret, frame = cam.read()
    
    # Create a copy of the frame to use in the functions
    frame_copy = frame.copy()
    
    # Grab the ROI from the original frame
    # using the top, bottom, left, and right specifications defined above
    # then create a grayscale version of roi; convert from BGR to GRAY
    # Recall than using OpenCV, the video frames are BGR when first loaded
    roi = frame[ROI_Top:ROI_Bottom,ROI_Right:ROI_Left]
    gray = cv2.cvtColor(roi,cv2.COLOR_BGR2GRAY)
    
    # Next, apply a Gaussian blur to the grayscale image to help average all the values
    # use a 7x7 kernel and 
    gray = cv2.GaussianBlur(gray,(7,7),0)
    
    # Now, define the number of frames to be used for the ROI definition
    if num_frames < 60:
        calc_accum_avg(gray,accumulated_weight)
        
        # DETERMINE ROI (ALSO WINDOW 1 - OVERALL LIVE STREAM)
        # While the background is being calculated, show a message
        # noting that the background is being calculated
        if num_frames <= 59:
            cv2.putText(frame_copy,'WAIT, GETTING BACKGROUND',(200,300),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
            cv2.imshow('Herb''s Finger Count Program',frame_copy)
    else:
        
        # HAND SEGMENT DETECTION AND CALCULATION
        # Otherwise, scan the hand, determine the segments passing the grayscale image:
        
        # First segment the hand region
        # The way the segment function was defined above, we cannot use tupple unpacking
        # if there is no hand, the parameter returned from segment is just "none"
        # So, we need to check for this first
        hand = segment(gray)
        
        # See if the hand was actually detected as an external contour
        # then unpack.
        # This means that if there is actually a hand detected, the information returned
        # by the segment function will actually contain a tuple; so, now we can use
        # tuple unpacking
        if hand is not None:
            thresholded_image, hand_segment = hand
            
            # WINDOW 1
            # Highlight the hand in the real image.
            # That is, draw the contours around the real hand in the live stream
            cv2.drawContours(frame_copy,[hand_segment+(ROI_Right,ROI_Top)],-1,(255,0,0),5)
            
            # Now, count the fingers
            fingers = count_fingers(thresholded_image, hand_segment)
            
            # Display the count on the live stream
            cv2.putText(frame_copy,str(fingers),(200,100),cv2.FONT_HERSHEY_SIMPLEX,3,(0,255,0),6)
            
            # WINDOW 2
            # Also show the thresholded image
            # create a new window
            # This window helps to view the process and decide if the threshold minimum value needs tuning
            # threshold_min = 25 at the start
            cv2.imshow('Thresholed Image',thresholded_image)
            
    # Draw a rectangle of the ROI
    cv2.rectangle(frame_copy,(ROI_Left,ROI_Top),(ROI_Right,ROI_Bottom),(0,255,0),2)
    
    # Increment the number of frames
    num_frames += 1
    
    # Show the finger count on the live-stream window
    cv2.imshow('Herb''s Finger Count Program',frame_copy)
    
    # Add a means to exit the program using the escape key (27)
    k = cv2.waitKey(1) & 0xFF
    
    if k == 27:
        break
        
# Release the camera stream and close all the windows if the ESC key is pressed
cam.release()
cv2.destroyAllWindows()
    
    

AttributeError: 'NoneType' object has no attribute 'copy'